# notes

all using pixelwise loss and equivalent scheduler params 

In [13]:
# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [14]:
from pathlib import Path

data_dir = Path("./data")
data_fpath = data_dir / "2022-07-bmvc-report-01-data03-sweep-03-prepro-dropout-00-partial-01.csv"

import pandas as pd
df = pd.read_csv(data_fpath) 

def check_df(df):
    print(df.shape)
    print(df.dtypes)
    return df.head(5)

check_df(df)

(855, 10)
Name                           object
normal_class_fullqualified     object
supervise_mode                 object
State                          object
dropout_parameters            float64
preprocessing                  object
test/avg-precision            float64
test/roc-auc                  float64
mvtec_class_type               object
loss                           object
dtype: object


,Name,normal_class_fullqualified,supervise_mode,State,dropout_parameters,preprocessing,test/avg-precision,test/roc-auc,mvtec_class_type,loss
0,earnest-sweep-57,mvtec_00_bottle,synthetic-anomaly-confetti,finished,0.3,lcnaug2,0.234132,0.891439,object,pixelwise-batch-avg
1,fine-sweep-63,mvtec_00_bottle,synthetic-anomaly-confetti,finished,0.3,lcnaug2,0.236333,0.892664,object,pixelwise-batch-avg
2,worldly-sweep-65,mvtec_00_bottle,synthetic-anomaly-confetti,finished,0.3,lcnaug2,0.196549,0.871634,object,pixelwise-batch-avg
3,rural-sweep-61,mvtec_00_bottle,synthetic-anomaly-confetti,finished,0.3,lcnaug2,0.216112,0.882333,object,pixelwise-batch-avg
4,solar-sweep-41,mvtec_00_bottle,synthetic-anomaly-confetti,finished,0.1,lcnaug2,0.215895,0.882238,object,pixelwise-batch-avg


In [15]:
data_df = pd.read_csv(data_fpath) 
df = data_df
df.drop(columns=["State"], inplace=True)
check_df(df)

(855, 9)
Name                           object
normal_class_fullqualified     object
supervise_mode                 object
dropout_parameters            float64
preprocessing                  object
test/avg-precision            float64
test/roc-auc                  float64
mvtec_class_type               object
loss                           object
dtype: object


,Name,normal_class_fullqualified,supervise_mode,dropout_parameters,preprocessing,test/avg-precision,test/roc-auc,mvtec_class_type,loss
0,earnest-sweep-57,mvtec_00_bottle,synthetic-anomaly-confetti,0.3,lcnaug2,0.234132,0.891439,object,pixelwise-batch-avg
1,fine-sweep-63,mvtec_00_bottle,synthetic-anomaly-confetti,0.3,lcnaug2,0.236333,0.892664,object,pixelwise-batch-avg
2,worldly-sweep-65,mvtec_00_bottle,synthetic-anomaly-confetti,0.3,lcnaug2,0.196549,0.871634,object,pixelwise-batch-avg
3,rural-sweep-61,mvtec_00_bottle,synthetic-anomaly-confetti,0.3,lcnaug2,0.216112,0.882333,object,pixelwise-batch-avg
4,solar-sweep-41,mvtec_00_bottle,synthetic-anomaly-confetti,0.1,lcnaug2,0.215895,0.882238,object,pixelwise-batch-avg


In [16]:
data_df = df

In [17]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore
import matplotlib as mpl
import matplotlib.cm as cm

def get_color_mask_compare_by_line_every_n_cols(pt, n):
        
    assert pt.shape[1] % n == 0, f"pt.shape[1] % n != 0: {pt.shape[1] % n}"

    def get_ordering_signal(subdf: DataFrame) -> DataFrame:
        """at each line, the values are assigned a value from 0 to 1 respecting the ordering, such that 0 is the min and 1 is the max"""
        # it's ok to round up to 3 decimals because i will probably never have more than 1001 values
        # and it makes the table easier to read/debug
        
        if subdf.isna().any(axis=None):
            return DataFrame(-1 * np.ones_like(subdf.values), columns=subdf.columns, index=subdf.index).astype(int)
        
        subdf = subdf.apply(
            lambda series: series.apply(lambda x: percentileofscore(np.sort(series.values), x, kind="strict") / 100), 
            axis=1
        )
        # 1/n is the min value of percentileofscore
        return (subdf / (1 / n)).round(decimals=0).astype(int)

    return np.concatenate([
        np.concatenate(
            [   
                get_ordering_signal(pt.iloc[rowidx:(rowidx + 1), i * n:(i + 1) * n]).values
                for i in range(pt.shape[1] // n)
            ],
            axis=1,
        )
        for rowidx in range(pt.shape[0])
    ], axis=0)
    
COLOR_MINUS_1 = "background-color: #00000000"
alpha = "55"
COLOR_0 = f'background-color: #DC143C{alpha}'
COLOR_MIDDLE = f'background-color: #FFFFFF{alpha}'
COLOR_1 = f'background-color: #228B22{alpha}'

def color_from_mask(color_mask: np.ndarray):
    n = color_mask.max() + 1
    assert n > 1, f"n > 1: {n}"
    
    if n == 2:
        def color(_):
            colormap = np.empty_like(color_mask, dtype=object)
            colormap[color_mask == -1] = COLOR_MINUS_1
            colormap[color_mask == 0] = COLOR_0
            colormap[color_mask == 1] = COLOR_1
            return colormap    
        return color
    
    elif n == 3:
        def color(_):
            colormap = np.empty_like(color_mask, dtype=object)
            colormap[color_mask == -1] = COLOR_MINUS_1
            colormap[color_mask == 0] = COLOR_0
            colormap[color_mask == 1] = COLOR_MIDDLE
            colormap[color_mask == 2] = COLOR_1
            return colormap
        return color
    
    elif n >= 4:
        def color(_):
            colormap = np.empty_like(color_mask, dtype=object)
            colormap[color_mask == -1] = COLOR_MINUS_1
            
            to_rgba = cm.ScalarMappable(
                norm=mpl.colors.Normalize(vmin=0, vmax=(n-1)), 
                cmap=cm.get_cmap("bwr_r")
            ).to_rgba
            
            def rgba2str(rgba):
                rgba = (255 * np.array(rgba)).astype(int)
                return f"background-color: #{rgba[0]:02x}{rgba[1]:02x}{rgba[2]:02x}{alpha}"

            import itertools
            size_i, size_j = color_mask.shape
            
            # i got lazy and used loops
            for idxi, idxj in itertools.product(range(size_i), range(size_j)):
                val = color_mask[idxi, idxj]
                if val == -1:
                    colormap[idxi, idxj] = COLOR_MINUS_1
                    continue
                colormap[idxi, idxj] = rgba2str(to_rgba(val))
            return colormap
        return color
    else:
        raise NotImplementedError(f"n > 3 is not implemented, found {n}")

In [18]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from functools import partial

df = data_df.copy()
	
row_cols = ["mvtec_class_type", "normal_class_fullqualified"]
column_cols = ["supervise_mode", "preprocessing", "dropout_parameters"]

metrics = {
    "test/avg-precision": ["mean", "std"],
    # "validate/avg-precision-histavg": ["mean",],
    # "test/roc-auc": ["mean", "std"],
    # "validate/roc-auc-histavg": ["mean",],
}
metric_cols = [(k, v) for k, list_vals in metrics.items() for v in list_vals]

pt_all_metrics = pd.pivot_table(df, index=row_cols, columns=column_cols, aggfunc=metrics)

# when it's a percentage
pt_all_metrics = 100 * pt_all_metrics

pt_test_avg = pt_all_metrics[("test/avg-precision", "mean")]
# pt_test_avg = pt_all_metrics[("test/roc-auc", "mean")]

pt_test_avg.loc[("", "object mean"), :] = pt_test_avg.loc["object"].mean(axis=0)
pt_test_avg.loc[("", "texture mean"), :] = pt_test_avg.loc["texture"].mean(axis=0)
pt_test_avg.loc[("", "all mean"), :] = pt_test_avg.mean(axis=0)

pt_test_std = pt_all_metrics[("test/avg-precision", "std")]
# # pt_test_std = pt_all_metrics[("test/roc-auc", "std")]

pt_test_std.loc[("", "object mean"), :] = pt_test_std[~pt_test_std.isna()].loc["object"].mean(axis=0)
pt_test_std.loc[("", "texture mean"), :] = pt_test_std[~pt_test_std.isna()].loc["texture"].mean(axis=0)
pt_test_std.loc[("", "all mean"), :] = pt_test_std[~pt_test_std.isna()].mean(axis=0)

# pt_validate_histmean_avg = pt_all_metrics[("validate/avg-precision-histavg", "mean")]
# pt_validate_histmean_avg = pt_all_metrics[("validate/roc-auc-histavg", "mean")]

# pt_validate_histmean_avg.loc[("", "object mean"), :] = pt_validate_histmean_avg.loc["object"].mean(axis=0)
# pt_validate_histmean_avg.loc[("", "texture mean"), :] = pt_validate_histmean_avg.loc["texture"].mean(axis=0)
# pt_validate_histmean_avg.loc[("", "all mean"), :] = pt_validate_histmean_avg.mean(axis=0)

pt_composed = pt_test_avg.applymap("{:.1f}".format) + " " + pt_test_std.applymap("({:.1f})".format) # + " " + pt_validate_histmean_avg.applymap("[{:.1f}]".format)
pt_composed

/tmp/ipykernel_65067/2041565868.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_test_avg.loc[("", "object mean"), :] = pt_test_avg.loc["object"].mean(axis=0)
/tmp/ipykernel_65067/2041565868.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_test_std.loc[("", "object mean"), :] = pt_test_std[~pt_test_std.isna()].loc["object"].mean(axis=0)


supervise_mode                              real-anomaly               \
preprocessing                                    lcnaug1                
dropout_parameters                                  0.00         0.03   
mvtec_class_type normal_class_fullqualified                             
object           mvtec_00_bottle              65.2 (8.0)   65.2 (8.0)   
                 mvtec_01_cable               59.2 (2.8)   60.8 (3.8)   
                 mvtec_02_capsule             13.7 (5.1)   14.4 (5.0)   
                 mvtec_05_hazelnut           50.1 (16.0)    nan (nan)   
                 mvtec_07_metal_nut           85.9 (0.8)   85.9 (0.9)   
                 mvtec_08_pill                64.6 (8.5)   62.0 (6.2)   
                 mvtec_09_screw                6.0 (3.3)    6.3 (2.0)   
                 mvtec_11_toothbrush         36.8 (19.8)  49.2 (11.0)   
                 mvtec_12_transistor          54.6 (7.1)   54.9 (6.7)   
                 mvtec_14_zipper              66.1 (1.4)    nan (nan)   
texture          mvtec_03_carpet              72.0 (1.3)   70.0 (4.3)   
                 mvtec_04_grid                20.1 (1.5)   19.5 (1.7)   
                 mvtec_06_leather             41.4 (1.6)   42.8 (3.8)   
                 mvtec_10_tile                89.4 (2.3)   90.2 (2.3)   
                 mvtec_13_wood               51.5 (10.1)   47.6 (9.9)   
                 object mean                  50.2 (7.3)   49.8 (5.5)   
                 texture mean                 54.9 (3.3)   54.1 (4.4)   
                 all mean                     51.9 (5.9)   51.5 (5.0)   

supervise_mode                                                         \
preprocessing                                                           
dropout_parameters                                  0.10         0.30   
mvtec_class_type normal_class_fullqualified                             
object           mvtec_00_bottle              67.6 (6.8)   65.0 (9.1)   
                 mvtec_01_cable               61.0 (3.7)   60.5 (3.3)   
                 mvtec_02_capsule             12.6 (4.7)   13.6 (4.7)   
                 mvtec_05_hazelnut            44.7 (nan)  47.7 (14.9)   
                 mvtec_07_metal_nut           85.7 (0.8)   85.4 (0.4)   
                 mvtec_08_pill               61.1 (10.6)   63.3 (9.8)   
                 mvtec_09_screw                5.8 (1.9)    nan (nan)   
                 mvtec_11_toothbrush         36.8 (19.8)  36.8 (19.8)   
                 mvtec_12_transistor          54.9 (6.7)   54.0 (8.6)   
                 mvtec_14_zipper               nan (nan)    nan (nan)   
texture          mvtec_03_carpet              72.7 (1.6)   70.0 (4.3)   
                 mvtec_04_grid                19.8 (1.5)   19.3 (1.3)   
                 mvtec_06_leather             43.9 (3.7)   43.9 (3.7)   
                 mvtec_10_tile                90.0 (2.3)   89.8 (2.7)   
                 mvtec_13_wood                48.8 (9.8)   48.2 (8.6)   
                 object mean                  47.8 (6.9)   53.3 (8.8)   
                 texture mean                 55.1 (3.8)   54.3 (4.1)   
                 all mean                     50.5 (5.6)   53.7 (6.9)   

supervise_mode                                                        \
preprocessing                                    lcnaug2               
dropout_parameters                                  0.00        0.03   
mvtec_class_type normal_class_fullqualified                            
object           mvtec_00_bottle              68.7 (5.2)  69.2 (5.9)   
                 mvtec_01_cable               61.7 (4.2)  59.8 (2.2)   
                 mvtec_02_capsule             10.5 (4.2)  14.9 (6.7)   
                 mvtec_05_hazelnut           54.0 (21.2)   nan (nan)   
                 mvtec_07_metal_nut           89.6 (1.1)  89.6 (1.1)   
                 mvtec_08_pill                82.9 (3.6)  83.5 (2.7)   
                 mvtec_09_screw               10.6 (3.4)   9.7 (1.

# all: within same setup + preprocessing (dropout value compared)

In [19]:
pt = pt_composed.copy()
pt_color_from = pt_test_avg.copy()

color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=4)
pt.style.apply(color_from_mask(color_mask), axis=None)

# all: within same setup + dropout value (preprocessing compared)

In [20]:
neworder = [0, 2, 1]

def reorder_columns(df_):
    level_names = np.array(df_.columns.names)[neworder].tolist()
    df_ = df_.reorder_levels(neworder, axis=1)
    df_.columns = pd.MultiIndex.from_tuples(np.sort(df_.columns.values), names=level_names)
    return df_

pt = reorder_columns(pt_composed.copy())
pt_color_from = reorder_columns(pt_test_avg.copy())

color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

unsupervised: 3% dropout is the best, and `lcnaug2` is better and in the case 0% dropout the preprocessing is about the same; some improvement so far

semi-supervised: very high variances, it's hard to say which one is better; comparing the preprocessing in each dropout case is also hard to draw any conclusion about parameters choice